In [1]:

from BenchmarkProblems.RoyalRoad import RoyalRoad
from Core.get_pRef import get_pRef_from_metaheuristic

problem = RoyalRoad(5)

pRef = get_pRef_from_metaheuristic(problem=problem,
                                   sample_size=10000,
                                   which_algorithm="GA",
                                   unique=True)
print(pRef)

PRef with 907 samples, mean = 8.11


In [2]:


from Core.PS import PS
from Core.PSMetric.Linkage.TraditionalPerturbationLinkage import TraditionalPerturbationLinkage

pattern_1 = PS.from_string("1"*4 + "*" * 16)
pattern_2 = PS.from_string("1"*2 + "*"*2 +"1"*2 + "*"*2 + "*" * 12)
to_explain = pRef.get_best_solution()



atomicity_metric = TraditionalPerturbationLinkage(problem)
atomicity_metric.set_solution(to_explain)


def atomicity(ps):
    return -atomicity_metric.get_atomicity(ps)


def dependence(ps):
    return atomicity_metric.get_dependence(ps)

print( atomicity(pattern_1), dependence(pattern_1))
print( atomicity(pattern_2), dependence(pattern_2))

-4.0 0.0
-0.0 4.0


In [3]:
from SimplifiedSystem.ps_search_utils import run_pymoo_algorithm_with_checks
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.mutation.bitflip import BitflipMutation
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover
from SimplifiedSystem.Operators.Sampling import LocalPSGeometricSampling
from SimplifiedSystem.LocalPSSearchTask import LocalPSSearchTask

objectives = [atomicity, dependence]
population_size = 100

ga_problem = LocalPSSearchTask(solution_to_explain=to_explain,
                                objectives=objectives,
                                # unexplained_mask=None,
                                proportion_unexplained_that_needs_used=0.01,
                                proportion_used_that_should_be_unexplained=0.01)

# if there are no operators given, we have these defaults
sampling_operator = LocalPSGeometricSampling()
crossover_operator = SimulatedBinaryCrossover(prob=0.3)
mutation_operator = BitflipMutation(prob=1 / ga_problem.n_var)

# the next line of code is a bit odd, but it works! It uses a GA if there is one objective
algorithm = (GA if len(objectives) < 2 else NSGA2)(pop_size=population_size,
                                                   sampling=sampling_operator,
                                                   crossover=crossover_operator,
                                                   mutation=mutation_operator,
                                                   eliminate_duplicates=True)

pss = run_pymoo_algorithm_with_checks(pymoo_problem=ga_problem,
                                      algorithm=algorithm,
                                      reattempts_when_fail=5,
                                      ps_budget = 10000,
                                      verbose=True)

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       62 |     17 |  0.000000E+00 |  0.0080645161 |             - |             -
     2 |      162 |     19 |  0.000000E+00 |  0.000000E+00 |  0.1428571429 |         ideal
     3 |      262 |     22 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     4 |      362 |     22 |  0.000000E+00 |  0.000000E+00 |  0.2500000000 |         ideal
     5 |      462 |     24 |  0.000000E+00 |  0.000000E+00 |  0.1111111111 |         ideal
     6 |      562 |     24 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |      662 |      1 |  0.000000E+00 |  0.000000E+00 |  4.0000000000 |         nadir
     8 |      762 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |      862 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |      962 |      1 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f